# Gerekli kütüphaneler

In [1]:
import numpy as np
import pandas as pd
import re
import ast
import string
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(format = '%(asctime)s %(message)s',
                    datefmt = '%m/%d/%Y %I:%M:%S %p',
                    filename = 'logs/2_cleaning_data.log',
                    level=logging.DEBUG)

import warnings
warnings.filterwarnings('ignore')

# 1. Temizlenmemiş veriyi yükleyelim.

In [2]:
df_ = pd.read_csv('data/turkish-tv-series.csv', index_col=0)
logging.debug('Veri seti yüklendi.')
df = df_.copy()
df.head()

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Youtube,Facebook,Twitter,Instagram,Durumu
0,https://tr.wikipedia.org//w/index.php?title=Ads%C4%B1z_A%C5%9F%C4%B1klar&action=edit&redlink=1,Adsız Aşıklar,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?
1,https://tr.wikipedia.org//w/index.php?title=A%C4%9Flayan_Gelin&action=edit&redlink=1,Ağlayan Gelin,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?
2,https://tr.wikipedia.org//w/index.php?title=Aile_Alb%C3%BCm%C3%BC&action=edit&redlink=1,Aile Albümü,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?
3,https://tr.wikipedia.org//w/index.php?title=Al%C4%B1-Kara&action=edit&redlink=1,Alı-Kara,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?
4,https://tr.wikipedia.org//w/index.php?title=Altay_(dizi)&action=edit&redlink=1,Altay,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?


In [3]:
clean_df = df[['Dizi Linki', 'Dizi Adı']]
logging.debug('Temizlenmiş veriler için yeni veri seti oluşturuldu.')
clean_df.head()

,Dizi Linki,Dizi Adı
0,https://tr.wikipedia.org//w/index.php?title=Ads%C4%B1z_A%C5%9F%C4%B1klar&action=edit&redlink=1,Adsız Aşıklar
1,https://tr.wikipedia.org//w/index.php?title=A%C4%9Flayan_Gelin&action=edit&redlink=1,Ağlayan Gelin
2,https://tr.wikipedia.org//w/index.php?title=Aile_Alb%C3%BCm%C3%BC&action=edit&redlink=1,Aile Albümü
3,https://tr.wikipedia.org//w/index.php?title=Al%C4%B1-Kara&action=edit&redlink=1,Alı-Kara
4,https://tr.wikipedia.org//w/index.php?title=Altay_(dizi)&action=edit&redlink=1,Altay


# 2. Veri Temizleme

- Senarist
- Yönetmen
- Başrol
- Besteci
- Bölüm sayısı
- Yapımcı
- Gösterim süresi
- Yapım şirketi
- Kanal
- Platform
- Ses formatı

In [2]:
def remove_parentheses(text):
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub('1. sezon', '', text)
    text = re.sub('2. sezon', '', text)
    text = re.sub(r',{2,}', ',', text)
    #text = re.sub(',', '', text)
    text = text.strip()
    return text

In [5]:
clean_df['Tür'] = df['Tür']
logging.debug('Tür sütunu temizlendi.')

In [6]:
clean_df['Uyarlama'] = df['Uyarlama']
logging.debug('Uyarlama sütunu temizlendi.')

In [7]:
clean_df['Senarist'] = df['Senarist'].apply(remove_parentheses)
logging.debug('Senarist sütunu temizlendi.')

In [8]:
clean_df['Yönetmen'] = df['Yönetmen'].apply(remove_parentheses)
logging.debug('Yönetmen sütunu temizlendi.')

In [9]:
clean_df['Başrol'] = df['Başrol'].apply(remove_parentheses)
logging.debug('Başrol sütunu temizlendi.')

In [10]:
clean_df['Besteci'] = df['Besteci'].apply(remove_parentheses)
logging.debug('Besteci sütunu temizlendi.')

In [11]:
clean_df['Sezon sayısı'] = df['Sezon sayısı'].apply(remove_parentheses)
logging.debug('Sezon sayısı sütunu temizlendi.')

In [12]:
clean_df['Bölüm sayısı'] = df['Bölüm sayısı'].apply(remove_parentheses)
logging.debug('Bölüm sayısı sütunu temizlendi.')

In [13]:
clean_df['Yapımcı'] = df['Yapımcı'].apply(remove_parentheses)
logging.debug('Yapımcı sütunu temizlendi.')

In [14]:
clean_df['Mekân'] = df['Mekân']
logging.debug('Mekan sütunu temizlendi.')

In [15]:
clean_df['Görüntü yönetmeni'] = df['Görüntü yönetmeni'].apply(remove_parentheses)
logging.debug('Görüntü yönetmeni sütunu temizlendi.')

In [16]:
def extract_minute(text):
    if text != '?':
        text = remove_parentheses(text)
        if '-' in text:
            text = text.split('-')[0]
            text = text.strip()
            return text
        else:
            text = re.sub('dakika', '', text)
            text = re.sub(',', '', text)
            text = text.strip()
            return text
    else:
        return text.strip()

In [17]:
clean_df['Gösterim süresi'] = df['Gösterim süresi'].apply(extract_minute)
logging.debug('Gösterim süresi sütunu temizlendi.')

In [18]:
clean_df['Yapım şirketi'] = df['Yapım şirketi'].apply(remove_parentheses)
logging.debug('Yapım şirketi sütunu temizlendi.')

In [19]:
clean_df['Kanal'] = df['Kanal']
logging.debug('Kanal sütunu temizlendi.')

In [20]:
clean_df['Platform'] = df['Platform']
logging.debug('Platform sütunu temizlendi.')

In [21]:
clean_df['Ses formatı'] = df['Ses formatı']
logging.debug('Ses formatı sütunu temizlendi.')

In [22]:
clean_df['Yayın tarihi'] = df['Yayın tarihi']
logging.debug('Yayın tarihi sütunu temizlendi.')

In [23]:
def split_date(text):
    text = remove_parentheses(text)
    if text != '?':
        text = text.replace(',', '')
        text = text.split('-')[0]
        if text.startswith(':'):
            text = text.replace(':', '')
            text = text.strip()
        if len(text.strip().split(' ')) >= 3:
            year = text.split(' ')[2]
            year = year.strip()
        elif len(text.strip().split(' ')) >= 2:
            year = text.strip()
            year = text.split(' ')[1]
        else:
            year = text.strip()

        #print(text, '||', year)
        return year
        
    else:
        return text

In [24]:
clean_df['Tarih'] = df['Yayın tarihi'].apply(split_date)
logging.debug('Tarih sütunu temizlendi.')

In [25]:
clean_df['Youtube'] = df['Youtube']
logging.debug('Youtube sütunu temizlendi.')

In [26]:
clean_df['Facebook'] = df['Facebook']
logging.debug('Facebook sütunu temizlendi.')

In [27]:
clean_df['Twitter'] = df['Twitter']
logging.debug('Twitter sütunu temizlendi.')

In [28]:
clean_df['Instagram'] = df['Instagram']
logging.debug('Instagram sütunu temizlendi.')

In [29]:
clean_df['Durumu'] = df['Durumu']
logging.debug('Durum sütunu temizlendi.')

# 3. Sonuç

In [30]:
clean_df.head()

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
0,https://tr.wikipedia.org//w/index.php?title=Ads%C4%B1z_A%C5%9F%C4%B1klar&action=edit&redlink=1,Adsız Aşıklar,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?
1,https://tr.wikipedia.org//w/index.php?title=A%C4%9Flayan_Gelin&action=edit&redlink=1,Ağlayan Gelin,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?
2,https://tr.wikipedia.org//w/index.php?title=Aile_Alb%C3%BCm%C3%BC&action=edit&redlink=1,Aile Albümü,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?
3,https://tr.wikipedia.org//w/index.php?title=Al%C4%B1-Kara&action=edit&redlink=1,Alı-Kara,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?
4,https://tr.wikipedia.org//w/index.php?title=Altay_(dizi)&action=edit&redlink=1,Altay,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?


In [31]:
clean_df.shape

(1522, 25)

In [32]:
clean_df = clean_df[57:]
clean_df = clean_df.reset_index(drop=True)

In [33]:
logging.debug('Tarihi 2024 olan veriler temizlendi.')

In [34]:
clean_df.head()

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
0,https://tr.wikipedia.org//wiki/Ad%C4%B1m_Farah,Adım Farah,Dram,"La Chica que Limpia,–Greta Molas","Deniz Dargı,Cem Görgeç,Cenk Boğatur,Serdar Soydan,Toprak Karaoğlu,Seda Çalışır Karaoğlu,","Recai Karagöz,Adem Demir,","Demet Özdemir,Engin Akyürek",Aytekin Ataş,2,25,Saner Ayar,İstanbul,Veysel Tekşahin,140,O3 Medya,FOX,?,Stereo,1 Mart 2023 - 30 Aralık 2023,2023,1,1,1,1,Devam ediyor
1,https://tr.wikipedia.org//wiki/Aile_(dizi),Aile,Dram,?,"Hakan Bonomo,Ali Kobanbay,","Ahmet Katıksız,Gökçen Usta,","Kıvanç Tatlıtuğ,Serenay Sarıkaya",Toygar Işıklı,2,24,Kerem Çatay,"İstanbul,İzmir",?,120,Ay Yapım,Show TV,?,Stereo,7 Mart 2023 - günümüz,2023,0,0,1,1,Devam ediyor
2,https://tr.wikipedia.org//wiki/Akif_(dizi),Akif,Tarihî,?,"Uğur Uzunok,Nurullah Sevimli,Tacettin Girgin",Selahattin Sancaklı,"Fikret Kuşkan,Özge Borak",Yıldıray Gürgen,1,13,Raif İnan,"İzmit,,,Kocaeli",?,56,Kirli Kedi,?,tabii,Stereo,7 Mayıs 2023,2023,0,0,0,0,Yayımlandı
3,https://tr.wikipedia.org//w/index.php?title=Aktris_(dizi)&action=edit&redlink=1,Aktris,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,0,0,?
4,https://tr.wikipedia.org//wiki/Al_Sancak_(dizi),Al Sancak,"Aksiyon,Dram",?,"Atilla Engin,Mehmet Arı,Selman Kılıçaslan","Can Emre,Günay Günaydın,","Uğur Güneş,Gülsim Ali",Cem Öget,1,19,Mehmet Bozdağ,İstanbul,?,135,Bozdağ Film,TRT 1,?,Stereo,19 Ocak 2023 - 15 Haziran 2023,2023,1,1,1,1,Yayından kaldırıldı


In [35]:
clean_df.shape

(1465, 25)

In [36]:
clean_df = clean_df.sort_values(by='Tarih').reset_index(drop=True)
logging.debug('Veri seti tarihe göre sıralandı.')

In [37]:
clean_df.head()

,Dizi Linki,Dizi Adı,Tür,Uyarlama,Senarist,Yönetmen,Başrol,Besteci,Sezon sayısı,Bölüm sayısı,Yapımcı,Mekân,Görüntü yönetmeni,Gösterim süresi,Yapım şirketi,Kanal,Platform,Ses formatı,Yayın tarihi,Tarih,Youtube,Facebook,Twitter,Instagram,Durumu
0,https://tr.wikipedia.org//wiki/Kaynanalar,Kaynanalar,Komedi,?,?,"Tekin Akmansoy,Arzu Akmansoy","Tekin Akmansoy,Leman Çıdamlı,Defne Yalnız,Sevda Aydan,Ege Aydan,Münir Caner,Sermin Hürmeriç,Sezai Aydın",?,14,314,?,?,?,?,?,"TRT 1,Kanal D",?,?,19 Mayıs 1974 - 24 Ocak 2005,1974,0,0,0,0,Sona erdi
1,"https://tr.wikipedia.org//wiki/A%C5%9Fk-%C4%B1_Memnu_(dizi,_1975)",Aşk-ı Memnu,"Aile,Dram",?,Halit Refiğ,Halit Refiğ,"Suna Keskin,Müjde Ar,Şükran Güngör,Neriman Köksal,Itır Esen",?,1,6,?,İstanbul,?,?,?,TRT 1,?,?,19 Nisan 1975 - 24 Mayıs 1975,1975,0,0,0,0,Sona erdi
2,?,Kartallar Yüksek Uçar,Polisiye,?,Atilla İlhan,Hüseyin Karakaş,"Selda Alkor,Hüseyin Peyda,Serap Aksoy,Necmettin Çobanoğlu,Can Gürzap",?,1,3,?,?,?,?,?,?,?,?,26 Kasım 1983,1983,0,0,0,0,Yayından kaldırıldı
3,https://tr.wikipedia.org//wiki/%C3%9C%C3%A7_%C4%B0stanbul_(dizi),Üç İstanbul,Dram,?,Bülent Oran,Feyzi Tuna,"Ayda Aksel,Gülsen Tuncer,Celile Toyon,Burçin Oraloğlu,Nilgün Akçaoğlu",Timur Selçuk,1,?,Oskay Alptürk,?,Muzaffer Turan,?,Ercan Film,TRT 1,?,?,1983,1983,0,0,0,0,Sona erdi
4,"https://tr.wikipedia.org//wiki/K%C3%BC%C3%A7%C3%BCk_A%C4%9Fa_(dizi,_1984)",Küçük Ağa,"Dram,Aksiyon,Tarihî",?,Tarık Buğra,Yücel Çakmaklı,"Çetin Tekindor,Aydan Şener,Erol Taş,Fikret Hakan",?,1,8,İlksen Bektaş,?,Sineray,58,TRT,TRT 1,?,?,"25 Mart 1984,(,1984-03-25,),[1],-,13 Mayıs 1984,(,1984-05-13,),[2]",1984,0,0,0,0,Sona erdi


In [38]:
clean_df.to_csv('data/cleaned.csv')

In [39]:
logging.debug('Temizlenen veri seti kaydedildi.')